# Part I. ETL Pipeline for Pre-Processing the Files
## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES
### Import Python packages¶

In [1]:
# Import Python packages
# Python packages that are necesseary for csv reader, writer, append and cassandra connection (ETL pipeline)
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project.
## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. The **event_datafile_new.csv** contains the following columns:

* artist
* firstName of user
* gender of user
* item number in session
* last name of user
* length of the song
* level (paid or free song)
* location of the user
* sessionId
* song title
* userId

# Begin writing your Apache Cassandra code in the cells below

## Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster([
    'node-0-server.cassandra.autoip.dcos.thisdcos.directory','node-1-server.cassandra.autoip.dcos.thisdcos.directory', 'node-2-server.cassandra.autoip.dcos.thisdcos.directory'],
    port=9042)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

## Create Keyspace

In [10]:
# Create keyspace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }"""
)

except Exception as e:
    print(e)

## Set Keyspace

In [11]:
# Set session to the keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [12]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Table music_app_library is created with composite primary key as the requirement is to get the details of artist, song 
## and length based on sessionid and iteminsession.

query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## convert to integer and float is included in the execute query
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## SELECT statement to verify the data was entered into the table
## Query  to get the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "select artist, song, length from music_app_history WHERE sessionid=338 and iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## Query 2: Get the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## Table is created with composite primary key as the query is selected based on user id and sessionid
## Clustering column iteminsession with order by is requred to avoid performance issues and prevent ALLOW FILTERING
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(sessionid int, userid int, iteminsession int, artist text, song text, firstname text, lastname text, \
        PRIMARY KEY ((sessionid, userid), iteminsession)) with clustering order by (iteminsession asc)"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (sessionid, userid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

In [17]:
## SELECT statements based on userid and sessionid and the query returned in Order by of clustering columns.
query = "select artist, song, firstname, lastname from song_playlist_session WHERE userid = 10 and sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [18]:
## Query 3: Get every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'
## A new table song user details with clustering order as many users can listen to the same song.
query = "CREATE TABLE IF NOT EXISTS song_user_details"
query = query + "(userid int, song text, firstname text, lastname text, \
           PRIMARY KEY ((song), userid)) with clustering order by (userid asc)"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_details (userid, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

In [ ]:
## query the table with song name to get the list of the users who listened to the same song.
query = "select firstname, lastname from song_user_details WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)